opossum3.0 (http://opossum.cmmt.ubc.ca/oPOSSUM3/) のSSAを用いてTSS上流2kb以内で、既存のTFモチーフ(JASPAR)の中で統計的に有意なモチーフを抽出した。<br>
条件は<br>
background gene:All 29347 genes in the oPOSSUM database<br>
Conservation cutoff:0.40<br>
Matrix score threshold:85%<br>
Amount of upstream / downstream sequence:2000 / 0<br>
Sort results by:Z-score<br>
<br>
このコードでは候補TFの中でMⅡの時期にBDF1>>ICR,ddY(not ddY>>BDF1)となっている遺伝子を抽出した。<br>
抽出の条件は、<br>
① BDF1 ≧ 0.3(FPKM)<br>
② BDF1 >> ICR,ddY (log2FC ≧ 0.6、P-value < 0.05)<br>
②'not ddY >> ICR (log2FC ≧ 0.6、P-value < 0.05)<br>
<br>
出力は./TFdata/TFlist.tsv

In [2]:
#!/usr/bin/env python

In [3]:
import numpy as np
import pandas as pd
from matplotlib import pylab as plt
import seaborn as sns
%matplotlib inline

# 目盛り調整のticker追加
import matplotlib.ticker as ticker

In [4]:
#データの読み込み(opossumのデータ)
df = pd.read_csv('./TFdata/BDF1expressed_opossum.tsv',sep='\t')
df

,TF,JASPAR ID,Class,Family,Tax Group,IC,GC Content,Target gene hits,Target gene non-hits,Background gene hits,Background gene non-hits,Target TFBS hits,Target TFBS nucleotide rate,Background TFBS hits,Background TFBS nucleotide rate,Z-score,Fisher score
0,PPARG,MA0066.1,Zinc-coordinating,Hormone-nuclear Receptor,vertebrates,20.365,0.523,1,82,26,29321,1,0.000469,26,0.000042,13.278,2.611
1,SP1,MA0079.2,Zinc-coordinating,BetaBetaAlpha-zinc finger,vertebrates,11.129,0.820,45,38,10383,18964,156,0.036600,33030,0.026500,12.904,7.953
2,RREB1,MA0073.1,Zinc-coordinating,BetaBetaAlpha-zinc finger,vertebrates,22.278,0.623,3,80,726,28621,6,0.002810,783,0.001260,8.999,1.083
3,INSM1,MA0155.1,Zinc-coordinating,BetaBetaAlpha-zinc finger,vertebrates,14.862,0.667,19,64,4416,24931,34,0.009560,6458,0.006220,8.748,3.268
4,PPARG::RXRA,MA0065.2,Zinc-coordinating,Hormone-nuclear Receptor,vertebrates,11.663,0.547,14,69,3162,26185,22,0.007730,4036,0.004860,8.504,2.810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,REL,MA0101.1,Ig-fold,Rel,vertebrates,10.515,0.559,18,65,6719,22628,24,0.005620,10987,0.008820,-7.084,0.438
112,Gata1,MA0035.2,Zinc-coordinating,GATA,vertebrates,10.878,0.373,33,50,10620,18727,65,0.016800,24697,0.021800,-7.158,1.256
113,Foxa2,MA0047.2,Winged Helix-Turn-Helix,Forkhead,vertebrates,13.268,0.315,20,63,7477,21870,30,0.008440,13029,0.012500,-7.653,0.424
114,MAX,MA0058.1,Zipper-Type,Helix-Loop-Helix,vertebrates,12.685,0.571,8,75,4814,24533,10,0.002340,6592,0.005290,-8.426,0.028


In [5]:
#::を含む行を抽出
df2 = df[df['TF'].str.match('.*::.*', na=False)]

In [6]:
#::の後だけにする
df2['TF'] = df2['TF'].str.replace('.*::', '')
df2.head(10)

/Users/shigenseigyo/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
/Users/shigenseigyo/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,TF,JASPAR ID,Class,Family,Tax Group,IC,GC Content,Target gene hits,Target gene non-hits,Background gene hits,Background gene non-hits,Target TFBS hits,Target TFBS nucleotide rate,Background TFBS hits,Background TFBS nucleotide rate,Z-score,Fisher score
4,RXRA,MA0065.2,Zinc-coordinating,Hormone-nuclear Receptor,vertebrates,11.663,0.547,14,69,3162,26185,22,0.007730,4036,0.004860,8.504,2.810
14,Cebpa,MA0019.1,Zipper-Type,Leucine Zipper,vertebrates,11.652,0.483,11,72,2744,26603,15,0.004220,3264,0.003140,3.922,1.889
26,ARNT,MA0259.1,Zipper-Type,Helix-Loop-Helix,vertebrates,9.740,0.657,29,54,8187,21160,67,0.012600,18340,0.011800,1.480,2.327
29,VDR,MA0074.1,Zinc-coordinating,Hormone-nuclear Receptor,vertebrates,20.451,0.527,1,82,204,29143,1,0.000351,206,0.000248,1.204,0.819
36,Gata1,MA0140.1,Zipper-Type,Helix-Loop-Helix,vertebrates,11.297,0.451,11,72,3035,26312,13,0.005480,3666,0.005300,0.500,1.438
41,Ahr,MA0006.1,Zipper-Type,Helix-Loop-Helix,vertebrates,9.532,0.715,36,47,11078,18269,101,0.014200,29489,0.014200,-0.020,1.756
43,RAR_DR5,MA0159.1,Zinc-coordinating,Hormone-nuclear Receptor,vertebrates,16.004,0.535,2,81,566,28781,2,0.000797,595,0.000812,-0.194,0.738
46,Tcfe2a,MA0092.1,Zipper-Type,Helix-Loop-Helix,vertebrates,10.144,0.507,33,50,9650,19697,67,0.015700,19875,0.015900,-0.433,2.178
59,RXRA,MA0115.1,Zinc-coordinating,Hormone-nuclear Receptor,vertebrates,27.878,0.445,0,83,31,29316,0,0.000000,31,0.000042,-1.716,0.000
70,MAX,MA0059.1,Zipper-Type,Helix-Loop-Helix,vertebrates,14.237,0.602,5,78,1753,27594,5,0.001290,1972,0.001740,-2.299,0.584


In [7]:
df = pd.concat([df,df2])
df

,TF,JASPAR ID,Class,Family,Tax Group,IC,GC Content,Target gene hits,Target gene non-hits,Background gene hits,Background gene non-hits,Target TFBS hits,Target TFBS nucleotide rate,Background TFBS hits,Background TFBS nucleotide rate,Z-score,Fisher score
0,PPARG,MA0066.1,Zinc-coordinating,Hormone-nuclear Receptor,vertebrates,20.365,0.523,1,82,26,29321,1,0.000469,26,0.000042,13.278,2.611
1,SP1,MA0079.2,Zinc-coordinating,BetaBetaAlpha-zinc finger,vertebrates,11.129,0.820,45,38,10383,18964,156,0.036600,33030,0.026500,12.904,7.953
2,RREB1,MA0073.1,Zinc-coordinating,BetaBetaAlpha-zinc finger,vertebrates,22.278,0.623,3,80,726,28621,6,0.002810,783,0.001260,8.999,1.083
3,INSM1,MA0155.1,Zinc-coordinating,BetaBetaAlpha-zinc finger,vertebrates,14.862,0.667,19,64,4416,24931,34,0.009560,6458,0.006220,8.748,3.268
4,PPARG::RXRA,MA0065.2,Zinc-coordinating,Hormone-nuclear Receptor,vertebrates,11.663,0.547,14,69,3162,26185,22,0.007730,4036,0.004860,8.504,2.810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,Tcfe2a,MA0092.1,Zipper-Type,Helix-Loop-Helix,vertebrates,10.144,0.507,33,50,9650,19697,67,0.015700,19875,0.015900,-0.433,2.178
59,RXRA,MA0115.1,Zinc-coordinating,Hormone-nuclear Receptor,vertebrates,27.878,0.445,0,83,31,29316,0,0.000000,31,0.000042,-1.716,0.000
70,MAX,MA0059.1,Zipper-Type,Helix-Loop-Helix,vertebrates,14.237,0.602,5,78,1753,27594,5,0.001290,1972,0.001740,-2.299,0.584
83,TCF3,MA0091.1,Zipper-Type,Helix-Loop-Helix,vertebrates,14.070,0.453,10,73,3554,25793,11,0.003090,4404,0.004240,-3.687,0.585


In [8]:
#::の前だけにする
df['TF'] = df['TF'].str.replace('::.*', '')
df

/Users/shigenseigyo/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


,TF,JASPAR ID,Class,Family,Tax Group,IC,GC Content,Target gene hits,Target gene non-hits,Background gene hits,Background gene non-hits,Target TFBS hits,Target TFBS nucleotide rate,Background TFBS hits,Background TFBS nucleotide rate,Z-score,Fisher score
0,PPARG,MA0066.1,Zinc-coordinating,Hormone-nuclear Receptor,vertebrates,20.365,0.523,1,82,26,29321,1,0.000469,26,0.000042,13.278,2.611
1,SP1,MA0079.2,Zinc-coordinating,BetaBetaAlpha-zinc finger,vertebrates,11.129,0.820,45,38,10383,18964,156,0.036600,33030,0.026500,12.904,7.953
2,RREB1,MA0073.1,Zinc-coordinating,BetaBetaAlpha-zinc finger,vertebrates,22.278,0.623,3,80,726,28621,6,0.002810,783,0.001260,8.999,1.083
3,INSM1,MA0155.1,Zinc-coordinating,BetaBetaAlpha-zinc finger,vertebrates,14.862,0.667,19,64,4416,24931,34,0.009560,6458,0.006220,8.748,3.268
4,PPARG,MA0065.2,Zinc-coordinating,Hormone-nuclear Receptor,vertebrates,11.663,0.547,14,69,3162,26185,22,0.007730,4036,0.004860,8.504,2.810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,Tcfe2a,MA0092.1,Zipper-Type,Helix-Loop-Helix,vertebrates,10.144,0.507,33,50,9650,19697,67,0.015700,19875,0.015900,-0.433,2.178
59,RXRA,MA0115.1,Zinc-coordinating,Hormone-nuclear Receptor,vertebrates,27.878,0.445,0,83,31,29316,0,0.000000,31,0.000042,-1.716,0.000
70,MAX,MA0059.1,Zipper-Type,Helix-Loop-Helix,vertebrates,14.237,0.602,5,78,1753,27594,5,0.001290,1972,0.001740,-2.299,0.584
83,TCF3,MA0091.1,Zipper-Type,Helix-Loop-Helix,vertebrates,14.070,0.453,10,73,3554,25793,11,0.003090,4404,0.004240,-3.687,0.585


In [9]:
#_の前だけにする
df['TF'] = df['TF'].str.replace('_.*', '')
df

/Users/shigenseigyo/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


,TF,JASPAR ID,Class,Family,Tax Group,IC,GC Content,Target gene hits,Target gene non-hits,Background gene hits,Background gene non-hits,Target TFBS hits,Target TFBS nucleotide rate,Background TFBS hits,Background TFBS nucleotide rate,Z-score,Fisher score
0,PPARG,MA0066.1,Zinc-coordinating,Hormone-nuclear Receptor,vertebrates,20.365,0.523,1,82,26,29321,1,0.000469,26,0.000042,13.278,2.611
1,SP1,MA0079.2,Zinc-coordinating,BetaBetaAlpha-zinc finger,vertebrates,11.129,0.820,45,38,10383,18964,156,0.036600,33030,0.026500,12.904,7.953
2,RREB1,MA0073.1,Zinc-coordinating,BetaBetaAlpha-zinc finger,vertebrates,22.278,0.623,3,80,726,28621,6,0.002810,783,0.001260,8.999,1.083
3,INSM1,MA0155.1,Zinc-coordinating,BetaBetaAlpha-zinc finger,vertebrates,14.862,0.667,19,64,4416,24931,34,0.009560,6458,0.006220,8.748,3.268
4,PPARG,MA0065.2,Zinc-coordinating,Hormone-nuclear Receptor,vertebrates,11.663,0.547,14,69,3162,26185,22,0.007730,4036,0.004860,8.504,2.810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,Tcfe2a,MA0092.1,Zipper-Type,Helix-Loop-Helix,vertebrates,10.144,0.507,33,50,9650,19697,67,0.015700,19875,0.015900,-0.433,2.178
59,RXRA,MA0115.1,Zinc-coordinating,Hormone-nuclear Receptor,vertebrates,27.878,0.445,0,83,31,29316,0,0.000000,31,0.000042,-1.716,0.000
70,MAX,MA0059.1,Zipper-Type,Helix-Loop-Helix,vertebrates,14.237,0.602,5,78,1753,27594,5,0.001290,1972,0.001740,-2.299,0.584
83,TCF3,MA0091.1,Zipper-Type,Helix-Loop-Helix,vertebrates,14.070,0.453,10,73,3554,25793,11,0.003090,4404,0.004240,-3.687,0.585


In [10]:
#drop duplicate
df = df.drop_duplicates(subset=['TF'])

In [11]:
#TFの先頭だけ大文字にして遺伝子名にする
df['gene_id'] = df['TF'].str.capitalize()
df

/Users/shigenseigyo/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,TF,JASPAR ID,Class,Family,Tax Group,IC,GC Content,Target gene hits,Target gene non-hits,Background gene hits,Background gene non-hits,Target TFBS hits,Target TFBS nucleotide rate,Background TFBS hits,Background TFBS nucleotide rate,Z-score,Fisher score,gene_id
0,PPARG,MA0066.1,Zinc-coordinating,Hormone-nuclear Receptor,vertebrates,20.365,0.523,1,82,26,29321,1,0.000469,26,0.000042,13.278,2.611,Pparg
1,SP1,MA0079.2,Zinc-coordinating,BetaBetaAlpha-zinc finger,vertebrates,11.129,0.820,45,38,10383,18964,156,0.036600,33030,0.026500,12.904,7.953,Sp1
2,RREB1,MA0073.1,Zinc-coordinating,BetaBetaAlpha-zinc finger,vertebrates,22.278,0.623,3,80,726,28621,6,0.002810,783,0.001260,8.999,1.083,Rreb1
3,INSM1,MA0155.1,Zinc-coordinating,BetaBetaAlpha-zinc finger,vertebrates,14.862,0.667,19,64,4416,24931,34,0.009560,6458,0.006220,8.748,3.268,Insm1
5,MZF1,MA0057.1,Zinc-coordinating,BetaBetaAlpha-zinc finger,vertebrates,9.400,0.588,37,46,10602,18745,126,0.029500,30104,0.024200,7.202,2.659,Mzf1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,Ahr,MA0006.1,Zipper-Type,Helix-Loop-Helix,vertebrates,9.532,0.715,36,47,11078,18269,101,0.014200,29489,0.014200,-0.020,1.756,Ahr
43,RAR,MA0159.1,Zinc-coordinating,Hormone-nuclear Receptor,vertebrates,16.004,0.535,2,81,566,28781,2,0.000797,595,0.000812,-0.194,0.738,Rar
46,Tcfe2a,MA0092.1,Zipper-Type,Helix-Loop-Helix,vertebrates,10.144,0.507,33,50,9650,19697,67,0.015700,19875,0.015900,-0.433,2.178,Tcfe2a
83,TCF3,MA0091.1,Zipper-Type,Helix-Loop-Helix,vertebrates,14.070,0.453,10,73,3554,25793,11,0.003090,4404,0.004240,-3.687,0.585,Tcf3


In [12]:
#データの読み込み DEseq2
Bvd = pd.read_csv('./DEseq2_data/BDF1vsddY_lrt.tsv',sep='\t')
BvI = pd.read_csv('./DEseq2_data/BDF1vsICR_lrt.tsv',sep='\t')
Ivd = pd.read_csv('./DEseq2_data/ICR1vsddY_lrt.tsv',sep='\t')

In [13]:
#gene_idを列に変換
Bvd['gene_id'] = Bvd.index
BvI['gene_id'] = BvI.index
Ivd['gene_id'] = Ivd.index

In [14]:
#発現量データの読み込み
ex = pd.read_csv('./seqData/M2_expression_mm10.tsv',sep='\t')

In [15]:
#データの結合 & NAを含む行の削除
Bvd = pd.merge(Bvd,ex ,on = 'gene_id',how = 'outer')
Bvd = Bvd.dropna(how='any')

BvI = pd.merge(BvI,ex ,on = 'gene_id',how = 'outer')
BvI = BvI.dropna(how='any')

Ivd = pd.merge(Ivd,ex ,on = 'gene_id',how = 'outer')
Ivd = Ivd.dropna(how='any')

In [16]:
#BDF1 vs ddY (log2FCが0.6以上、P-valueが0.05未満、かつBDF1のFPKM値が0.3以上となる遺伝子の抽出)
B_d = Bvd[(Bvd['log2FoldChange'] >= 0.6) & (Bvd['pvalue'] < 0.05) & (Bvd['BDF1_FPKM_avr'] >= 0.3)]

#BDF1 vs ICR (log2FCが0.6以上、P-valueが0.05未満、かつBDF1のFPKM値が0.3以上となる遺伝子の抽出)
B_I = BvI[(BvI['log2FoldChange'] >= 0.6) & (BvI['pvalue'] < 0.05) & (BvI['BDF1_FPKM_avr'] >= 0.3)]

#ICR vs ddY (log2FCが-0.6以下、P-valueが0.05未満となる遺伝子の抽出)
I_d = Ivd[(Ivd['log2FoldChange'] <= -0.6) & (Ivd['pvalue'] < 0.05)]

In [17]:
I_d

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,gene_id,BDF1_TPM_avr,ICR_TPM_avr,ddY_TPM_avr,BDF1_FPKM_avr,ICR_FPKM_avr,ddY_FPKM_avr
42,5.925693,-4.049443,1.980325,12.290150,2.144015e-03,1.100815e-02,1500009C09Rik,0.640,0.000,0.125,0.245,0.000,0.050
59,258.539719,-1.321877,0.254732,43.323916,3.911390e-10,7.751223e-09,1700003M02Rik,6.010,6.555,16.350,2.310,2.600,6.330
64,489.151680,-1.160270,0.183628,42.714941,5.303570e-10,1.039572e-08,1700010I14Rik,18.960,9.425,21.080,7.295,3.750,8.180
66,10.269393,-2.025595,1.090013,18.600419,9.140506e-05,6.941870e-04,1700011H14Rik,0.000,0.360,1.455,0.000,0.140,0.565
73,47.183019,-1.568031,0.473998,19.276641,6.518244e-05,5.128801e-04,1700016K19Rik,1.940,2.380,7.060,0.740,0.945,2.740
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14333,291.067401,-0.635275,0.234506,7.538644,2.306769e-02,7.912769e-02,Zmat5,30.855,22.095,34.195,11.900,8.755,13.295
14364,23.772405,-1.622945,0.785672,20.239707,4.027203e-05,3.351814e-04,Zp4-ps,4.715,0.475,1.475,1.815,0.190,0.575
14374,126.010865,-0.932660,0.367468,8.212260,1.647139e-02,6.049027e-02,Zscan10,3.435,1.740,3.300,1.315,0.685,1.280
14386,46.243799,-1.433566,0.622852,12.972498,1.524256e-03,8.200132e-03,Zscan4f,2.750,0.545,1.525,1.050,0.220,0.595


In [18]:
#BDF1 >= 0.3 となるTFの数
num1 = BvI[Bvd['BDF1_FPKM_avr'] >= 0.3]
num2 = pd.merge(num1,df ,on = 'gene_id',how = 'outer')
num2 = num2.dropna(how='any')
len(num2)

57

In [19]:
num2

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,gene_id,BDF1_TPM_avr,ICR_TPM_avr,ddY_TPM_avr,...,Target gene hits,Target gene non-hits,Background gene hits,Background gene non-hits,Target TFBS hits,Target TFBS nucleotide rate,Background TFBS hits,Background TFBS nucleotide rate,Z-score,Fisher score
701,3330.964582,0.335652,0.094797,16.357141,2.806028e-04,1.878552e-03,Ahr,41.085,31.965,32.230,...,36.0,47.0,11078.0,18269.0,101.0,0.014200,29489.0,0.014200,-0.020,1.756
1013,935.676079,0.137848,0.143626,1.007844,6.041564e-01,7.815464e-01,Arid3a,10.685,9.540,10.250,...,34.0,49.0,12972.0,16375.0,133.0,0.018700,44220.0,0.021300,-3.729,0.276
1052,5911.406145,-0.234150,0.090563,8.485237,1.436992e-02,5.426669e-02,Arnt,71.940,83.060,82.465,...,36.0,47.0,11078.0,18269.0,101.0,0.014200,29489.0,0.014200,-0.020,1.756
1053,5911.406145,-0.234150,0.090563,8.485237,1.436992e-02,5.426669e-02,Arnt,71.940,83.060,82.465,...,29.0,54.0,8187.0,21160.0,67.0,0.012600,18340.0,0.011800,1.480,2.327
2430,218.817207,-0.044226,0.228467,1.093031,5.789638e-01,7.632613e-01,Creb1,1.415,1.430,1.615,...,22.0,61.0,5860.0,23487.0,31.0,0.005810,8288.0,0.005320,1.358,2.391
2502,8249.056470,0.164586,0.089183,3.471583,1.762607e-01,3.592795e-01,Ctcf,136.925,119.890,125.120,...,8.0,75.0,1504.0,27843.0,8.0,0.003560,1620.0,0.002470,4.493,2.764
2722,1418.975697,-0.363770,0.124492,9.869988,7.190504e-03,3.048309e-02,Ddit3,113.425,143.005,116.260,...,11.0,72.0,2744.0,26603.0,15.0,0.004220,3264.0,0.003140,3.922,1.889
3105,6748.411838,-0.118596,0.085937,6.666783,3.567192e-02,1.115289e-01,E2f1,139.580,148.700,159.425,...,20.0,63.0,5017.0,24330.0,25.0,0.004690,7629.0,0.004900,-0.661,2.724
3195,2499.909299,0.510813,0.114788,30.033783,3.007785e-07,3.882900e-06,Egr1,53.750,37.005,55.650,...,17.0,66.0,4053.0,25294.0,26.0,0.006700,5796.0,0.005120,4.555,2.808
3264,558.191584,-0.362517,0.160397,5.460086,6.521648e-02,1.768144e-01,Elk1,8.100,10.235,9.615,...,34.0,49.0,10524.0,18823.0,82.0,0.019200,23389.0,0.018800,0.657,1.630


In [18]:
#merge1(BDF1 >> ICR,ddYとなる遺伝子)
B_sp = pd.merge(B_d,B_I ,on = 'gene_id',how = 'inner')
B_sp = B_sp.rename(columns={'BDF1_FPKM_avr_y': 'BDF1_FPKM_avr','ICR_FPKM_avr_y': 'ICR_FPKM_avr','ddY_FPKM_avr_y': 'ddY_FPKM_avr'})
B_sp

,baseMean_x,log2FoldChange_x,lfcSE_x,stat_x,pvalue_x,padj_x,gene_id,BDF1_TPM_avr_x,ICR_TPM_avr_x,ddY_TPM_avr_x,...,lfcSE_y,stat_y,pvalue_y,padj_y,BDF1_TPM_avr_y,ICR_TPM_avr_y,ddY_TPM_avr_y,BDF1_FPKM_avr,ICR_FPKM_avr,ddY_FPKM_avr
0,47.559367,2.897749,0.528966,39.514610,2.627311e-09,4.621693e-08,1190007I07Rik,13.820,2.685,1.815,...,0.506409,39.514610,2.627311e-09,4.621693e-08,13.820,2.685,1.815,5.335,1.070,0.705
1,73.742471,0.746831,0.407748,7.102112,2.869432e-02,9.453443e-02,1700010D01Rik,22.120,10.450,12.755,...,0.412757,7.102112,2.869432e-02,9.453443e-02,22.120,10.450,12.755,8.555,4.155,4.945
2,17.984517,3.585155,0.951430,14.757068,6.245156e-04,3.801369e-03,1700029F12Rik,2.495,1.055,0.195,...,0.786155,14.757068,6.245156e-04,3.801369e-03,2.495,1.055,0.195,0.965,0.415,0.075
3,51.574611,1.396318,0.489920,16.076574,3.228616e-04,2.117190e-03,1700029P11Rik,6.645,1.805,2.475,...,0.500282,16.076574,3.228616e-04,2.117190e-03,6.645,1.805,2.475,2.550,0.720,0.955
4,1618.814577,1.055023,0.121606,118.406418,1.942554e-26,1.488637e-24,1700047I17Rik2,39.180,17.170,18.450,...,0.121844,118.406418,1.942554e-26,1.488637e-24,39.180,17.170,18.450,15.090,6.810,7.165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,15.819980,8.033086,1.676833,51.833512,5.552594e-12,1.384018e-10,Zkscan4,1.200,0.000,0.000,...,1.676833,51.833512,5.552594e-12,1.384018e-10,1.200,0.000,0.000,0.465,0.000,0.000
546,2189.292285,1.082349,0.106709,156.828433,8.813296e-35,1.049365e-32,Zmat4,48.490,21.340,22.430,...,0.106842,156.828433,8.813296e-35,1.049365e-32,48.490,21.340,22.430,18.620,8.480,8.710
547,23.772405,1.645392,0.666945,20.239707,4.027203e-05,3.351814e-04,Zp4-ps,4.715,0.475,1.475,...,0.759121,20.239707,4.027203e-05,3.351814e-04,4.715,0.475,1.475,1.815,0.190,0.575
548,46.243799,0.819667,0.583700,12.972498,1.524256e-03,8.200132e-03,Zscan4f,2.750,0.545,1.525,...,0.615852,12.972498,1.524256e-03,8.200132e-03,2.750,0.545,1.525,1.050,0.220,0.595


In [19]:
#merge2(ddY >> ICRとなる遺伝子を除く)
B_sp = pd.merge(B_sp,I_d ,on = 'gene_id',how = 'left')
B_sp = B_sp.rename(columns={'BDF1_FPKM_avr_y': 'BDF1_FPKM_avr','ICR_FPKM_avr_y': 'ICR_FPKM_avr','ddY_FPKM_avr_y': 'ddY_FPKM_avr'})
B_sp = B_sp[B_sp['lfcSE'].isna()]
B_sp = B_sp.fillna(0) #nanを埋める
B_sp

,baseMean_x,log2FoldChange_x,lfcSE_x,stat_x,pvalue_x,padj_x,gene_id,BDF1_TPM_avr_x,ICR_TPM_avr_x,ddY_TPM_avr_x,...,lfcSE,stat,pvalue,padj,BDF1_TPM_avr,ICR_TPM_avr,ddY_TPM_avr,BDF1_FPKM_avr,ICR_FPKM_avr,ddY_FPKM_avr
0,47.559367,2.897749,0.528966,39.514610,2.627311e-09,4.621693e-08,1190007I07Rik,13.820,2.685,1.815,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,73.742471,0.746831,0.407748,7.102112,2.869432e-02,9.453443e-02,1700010D01Rik,22.120,10.450,12.755,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,17.984517,3.585155,0.951430,14.757068,6.245156e-04,3.801369e-03,1700029F12Rik,2.495,1.055,0.195,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,51.574611,1.396318,0.489920,16.076574,3.228616e-04,2.117190e-03,1700029P11Rik,6.645,1.805,2.475,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1618.814577,1.055023,0.121606,118.406418,1.942554e-26,1.488637e-24,1700047I17Rik2,39.180,17.170,18.450,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,2113.098434,0.960279,0.115454,79.575403,5.253163e-18,2.245766e-16,Zfp956,66.715,39.070,33.580,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
544,179.997997,1.511297,0.302871,35.323699,2.135778e-08,3.312181e-07,Zim1,5.415,1.920,1.865,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
545,15.819980,8.033086,1.676833,51.833512,5.552594e-12,1.384018e-10,Zkscan4,1.200,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
546,2189.292285,1.082349,0.106709,156.828433,8.813296e-35,1.049365e-32,Zmat4,48.490,21.340,22.430,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
#データの結合 & NAを含む行の削除
B_sp = pd.merge(B_sp,df ,on = 'gene_id',how = 'outer')
B_sp = B_sp.dropna(how='any')
B_sp

,baseMean_x,log2FoldChange_x,lfcSE_x,stat_x,pvalue_x,padj_x,gene_id,BDF1_TPM_avr_x,ICR_TPM_avr_x,ddY_TPM_avr_x,...,Target gene hits,Target gene non-hits,Background gene hits,Background gene non-hits,Target TFBS hits,Target TFBS nucleotide rate,Background TFBS hits,Background TFBS nucleotide rate,Z-score,Fisher score
202,520.059151,0.699283,0.178599,29.557263,3.816998e-07,0.000005,Klf4,14.505,7.565,8.765,...,39.0,44.0,11678.0,17669.0,150.0,0.03510,39293.0,0.03150,4.258,2.199
364,86.279313,1.380786,0.449474,11.158160,3.776039e-03,0.017866,Sox2,3.515,1.635,1.325,...,3.0,80.0,997.0,28350.0,3.0,0.00105,1087.0,0.00131,-1.519,0.618


In [21]:
B_sp = pd.merge(B_sp,ex ,on = 'gene_id',how = 'left')
B_sp = B_sp.rename(columns={'BDF1_FPKM_avr_y': 'BDF1_FPKM_avr','ICR_FPKM_avr_y': 'ICR_FPKM_avr','ddY_FPKM_avr_y': 'ddY_FPKM_avr'})
B_sp

,baseMean_x,log2FoldChange_x,lfcSE_x,stat_x,pvalue_x,padj_x,gene_id,BDF1_TPM_avr_x,ICR_TPM_avr_x,ddY_TPM_avr_x,...,Background TFBS hits,Background TFBS nucleotide rate,Z-score,Fisher score,BDF1_TPM_avr_y,ICR_TPM_avr_y,ddY_TPM_avr_y,BDF1_FPKM_avr,ICR_FPKM_avr,ddY_FPKM_avr
0,520.059151,0.699283,0.178599,29.557263,3.816998e-07,0.000005,Klf4,14.505,7.565,8.765,...,39293.0,0.03150,4.258,2.199,14.505,7.565,8.765,5.555,3.005,3.400
1,86.279313,1.380786,0.449474,11.158160,3.776039e-03,0.017866,Sox2,3.515,1.635,1.325,...,1087.0,0.00131,-1.519,0.618,3.515,1.635,1.325,1.345,0.645,0.515


In [22]:
B_sp = B_sp.set_index('gene_id')
B_sp = B_sp[['TF','JASPAR ID','Class','Family','Tax Group','IC','GC Content','Target gene hits','Target gene non-hits','Background gene hits','Background gene non-hits','Target TFBS hits','Target TFBS nucleotide rate','Background TFBS hits','Background TFBS nucleotide rate','Z-score','Fisher score','BDF1_FPKM_avr','ICR_FPKM_avr','ddY_FPKM_avr']]
B_sp

,TF,JASPAR ID,Class,Family,Tax Group,IC,GC Content,Target gene hits,Target gene non-hits,Background gene hits,Background gene non-hits,Target TFBS hits,Target TFBS nucleotide rate,Background TFBS hits,Background TFBS nucleotide rate,Z-score,Fisher score,BDF1_FPKM_avr,ICR_FPKM_avr,ddY_FPKM_avr
gene_id,,,,,,,,,,,,,,,,,,,,
Klf4,Klf4,MA0039.2,Zinc-coordinating,BetaBetaAlpha-zinc finger,vertebrates,12.618,0.771,39.0,44.0,11678.0,17669.0,150.0,0.03510,39293.0,0.03150,4.258,2.199,5.555,3.005,3.400
Sox2,Sox2,MA0143.1,Other Alpha-Helix,High Mobility Group,vertebrates,12.951,0.361,3.0,80.0,997.0,28350.0,3.0,0.00105,1087.0,0.00131,-1.519,0.618,1.345,0.645,0.515


In [23]:
B_sp.to_csv('./TFdata/TFlist.csv', sep=',',index=None)

転写因子のリストは使用するためcsvに保存

In [24]:
df = df.set_index('gene_id')
df

,TF,JASPAR ID,Class,Family,Tax Group,IC,GC Content,Target gene hits,Target gene non-hits,Background gene hits,Background gene non-hits,Target TFBS hits,Target TFBS nucleotide rate,Background TFBS hits,Background TFBS nucleotide rate,Z-score,Fisher score
gene_id,,,,,,,,,,,,,,,,,
Pparg,PPARG,MA0066.1,Zinc-coordinating,Hormone-nuclear Receptor,vertebrates,20.365,0.523,1,82,26,29321,1,0.000469,26,0.000042,13.278,2.611
Sp1,SP1,MA0079.2,Zinc-coordinating,BetaBetaAlpha-zinc finger,vertebrates,11.129,0.820,45,38,10383,18964,156,0.036600,33030,0.026500,12.904,7.953
Rreb1,RREB1,MA0073.1,Zinc-coordinating,BetaBetaAlpha-zinc finger,vertebrates,22.278,0.623,3,80,726,28621,6,0.002810,783,0.001260,8.999,1.083
Insm1,INSM1,MA0155.1,Zinc-coordinating,BetaBetaAlpha-zinc finger,vertebrates,14.862,0.667,19,64,4416,24931,34,0.009560,6458,0.006220,8.748,3.268
Mzf1,MZF1,MA0057.1,Zinc-coordinating,BetaBetaAlpha-zinc finger,vertebrates,9.400,0.588,37,46,10602,18745,126,0.029500,30104,0.024200,7.202,2.659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ahr,Ahr,MA0006.1,Zipper-Type,Helix-Loop-Helix,vertebrates,9.532,0.715,36,47,11078,18269,101,0.014200,29489,0.014200,-0.020,1.756
Rar,RAR,MA0159.1,Zinc-coordinating,Hormone-nuclear Receptor,vertebrates,16.004,0.535,2,81,566,28781,2,0.000797,595,0.000812,-0.194,0.738
Tcfe2a,Tcfe2a,MA0092.1,Zipper-Type,Helix-Loop-Helix,vertebrates,10.144,0.507,33,50,9650,19697,67,0.015700,19875,0.015900,-0.433,2.178


In [25]:
df.to_csv('./TFdata/TFlist_nonTrim.csv', sep=',',index=None)